## Grupo 2

visualizar histórico en un período de tiempo 
Histórico por día 
Histórico por mes
Histórico por año 
(pensar filtros: por planta, por tipo de material, por LE y RA)

Parte a coordinar con el grupo 3
Inputs= step temporal (día, mes, año) 
Output= Cantidad de material histórico sobre la base de datos total


In [1]:
# from google.colab import files
# uploaded = files.upload()
# for item in uploaded.keys():
#  print('Se subio "{}"'.format(item))

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import time
from plotly.subplots import make_subplots

In [2]:
pesajes_df=pd.read_csv("pesajes-01-01-2019-31-12-2020_anonimizado.csv", low_memory = False)
pesajes_df["fecha"] = pd.to_datetime(pesajes_df["fecha"],format="%Y-%m-%d")
# pesajes_df["año"] = pd.DatetimeIndex(pesajes_df.fecha).year
# pesajes_df["mes"] = pd.DatetimeIndex(pesajes_df.fecha).month
# pesajes_df["día"] = pd.DatetimeIndex(pesajes_df.fecha).day
pesajes_df.head(10)

,fecha,etapa,bolson,camion,peso,cartonero,legacyId,predio,material
0,2019-12-02,R26,55.0,HVS 858,29,COMPA1987,3633,CORTEJARENA,NaN
1,2019-12-02,R26,28.0,HVS 858,93,COMPA687,3599,CORTEJARENA,NaN
2,2019-12-02,R26,61.0,HVS 858,35,COMPA2144,3638,CORTEJARENA,NaN
3,2019-12-02,R26,33.0,HVS 858,27,COMPA972,3608,CORTEJARENA,NaN
4,2019-12-02,R26,30.0,HVS 858,45,COMPA693,3603,CORTEJARENA,NaN
5,2019-12-02,R26,24.0,HVS 858,46,COMPA658,5625,CORTEJARENA,NaN
6,2019-12-02,E5,19.0,NaN,56,COMPA468,4986,SAAVEDRA,NaN
7,2019-12-02,E5,19.0,NaN,42,COMPA468,4986,SAAVEDRA,NaN
8,2019-12-02,E5,64.0,NaN,79,COMPA104,1342,SAAVEDRA,NaN
9,2019-12-02,E5,55.0,NaN,57,COMPA1834,1409,SAAVEDRA,NaN


In [19]:
def pesos_historico_predios(data):
    '''
    Grafica fecha contra peso (promedio o suma total) entregado ese día agrupando según el predio.
    Devuelve la figura de plotly conteniendo el gráfico.
    '''
    df = data.groupby(by=["fecha", "predio", "material"], as_index = False).sum()    # Para sacar la info de materiales, sacar "material"
    fig = px.bar(df, x="fecha", y="peso", color="predio", hover_data=["material"]) # "group"
    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])    # Ignora fin de semana
    fig.update_layout(bargap=0.01)
    return fig

pesos_historico_predios(pesajes_df).show()

In [4]:
def pesos_historico_materiales(data):
    '''
    Grafica fecha contra peso (promedio o suma total) entregado ese día agrupando según el predio.
    Devuelve la figura de plotly conteniendo el gráfico.
    '''
    df = data.groupby(by=["fecha", "material"], as_index = False).sum()
    fig = px.bar(df, x="fecha", y="peso", color="material", barmode = "group")
    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])    # Ignora fin de semana
    return fig

In [40]:
def pesos_historico_promedio(data): #quedo vieja, la reemplaza pesos_historico_promedio2

    '''
    Prueba. Grafica el "rolling average" (en cada dia muestra el promedio de los últimos 7 dias). 
    '''
    df = data.groupby(by=["fecha", "predio"], as_index = False).sum()
    #print(df)
    
    #recuperamos primer fecha y ultima fecha
    df= df.sort_values(by="fecha")
    
    primer_fecha=df["fecha"].iloc[0]
    ultima_fecha=df["fecha"].iloc[-1]
    rango_fechas=pd.date_range(primer_fecha, ultima_fecha)
    
    #agregamos fechas con cero en los dias que no entro material
    for predio in data["predio"].unique():
        for f in rango_fechas:
            df_aux=df[(df["fecha"]==f) & (df["predio"]==predio)]
            if len(df_aux)==0:
                df=df.append({"fecha":f, "predio": predio, "peso":0},ignore_index=True) #dejamos el bolson en nan
    
    df= df.sort_values(by="fecha")
    
    
    figures = []
    for predio in data["predio"].unique():
        print(predio)
        df_predio = df.loc[df['predio'] == predio]
        df_predio = df_predio.rolling("7d", on = "fecha", min_periods = 0).mean()
        #df_predio["peso"] = df_predio["peso"] / 7
        #print(df_predio)
        fig = px.line(df_predio, x = "fecha", y = "peso")
        #fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
        figures.append(fig)
    return figures

    
start_time = time.time()
    

figures = pesos_historico_promedio(pesajes_df)
figures[1].show()
print("--- %s seconds ---" % (time.time() - start_time))

CORTEJARENA
SAAVEDRA
nan
BARRACAS
AVELLANEDA


--- 11.881706953048706 seconds ---


In [30]:
#hace lo mismo que la funcion anterior pero con otro metodo de llenar los ceros. Ambas funcionan y ambas son lentas.

def pesos_historico_promedio_2(data):
    '''
    Prueba. Grafica el "rolling average" (en cada dia muestra el promedio de los últimos 7 dias). 
    '''
    df = data.groupby(by=["fecha", "predio"], as_index = False).sum()
    #print(df)
    
    #recuperamos primer fecha y ultima fecha
    df= df.sort_values(by="fecha")
    df=df[df['predio'].notna()]
    
    primer_fecha=df["fecha"].iloc[0]
    ultima_fecha=df["fecha"].iloc[-1]
    rango_fechas=pd.date_range(primer_fecha, ultima_fecha,freq="B") #B de business days
    
    
    figures = []
    for predio in df["predio"].unique():
        print(predio)
        df_predio = df.loc[df['predio'] == predio]
        
        #agrego ceros
        df_predio.set_index(keys="fecha",inplace=True)
        df_predio=df_predio.reindex(rango_fechas, fill_value=0)
        
        
        df_predio = df_predio.rolling("10d", min_periods = 0).sum()
        df_predio["peso"] = df_predio["peso"] / 10
        
        #fig = px.line(df_predio, x = "fecha", y = "peso")
        fig = px.line(df_predio, y = "peso")
        #fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
        figures.append(fig)
    return figures

#start_time = time.time()
figures = pesos_historico_promedio_2(pesajes_df)

figures[0].show()
#print("--- %s seconds ---" % (time.time() - start_time))

BARRACAS
CORTEJARENA
SAAVEDRA
AVELLANEDA


In [8]:
def torta(data):
    '''
    Realiza un gráfico de torta del peso por material.
    Devuelve la figura de plotly conteniendo el gráfico.
    '''
    df = data.groupby(by=["material"], as_index = False).sum()
    fig = px.pie(df, values="peso", names="material", title='')
    fig.update_traces(hoverinfo="label+percent", textfont_size=14,
                      textinfo="percent", marker=dict(line=dict(color="#FFFFFF",width=0.1)),
                      textposition="auto")
    return fig

torta(pesajes_df).show()

In [8]:
# Ejemplo de Métrica
def centro_principal(data):
    '''
    Devuelve una lista que contiene el nombre del predio que recolectó un mayor peso total.
    Si hay empate, devuelve multiples nombres.
    '''
    df = data.groupby(by=["predio"], as_index = False).sum()
    res = df[df.peso == df.peso.max()]["predio"].to_list()
    return res


In [9]:
import plotly.graph_objects as go

def tabla(data):
    
    df =data.groupby(by=["predio", "material"], as_index = False).sum()
    df = df.drop(columns = ["bolson"])
    print(df)
    
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(["predio", "material", "peso total (kg)"]),
                    fill_color='paleturquoise',
                    align='left'),
        
        cells=dict(values=[df["predio"], df["material"], df["peso"]],
                   fill_color='lavender',
                   align='left'))
    ])
    return fig
    
fig = tabla(pesajes_df)
fig.show()

        predio  material    peso
0   AVELLANEDA      TELA    6123
1     BARRACAS  Chatarra    4136
2     BARRACAS  Mezcla B  151714
3     BARRACAS  Vidrio B   31966
4  CORTEJARENA  Chatarra   11231
5  CORTEJARENA  Mezcla B  194126
6  CORTEJARENA      TELA   35466
7  CORTEJARENA  Vidrio B    9132
8     SAAVEDRA  Mezcla B  110116


In [23]:

def datos_tabla(data):
    df =data.groupby(by=["predio", "material"]).sum()
    print(df.head(10))
    df = df.drop(columns = ["bolson"])
    suma_predios = df.groupby(by=["predio"]).sum()
    suma_materiales = df.groupby(by=["material"]).sum()
    return df, suma_predios, suma_materiales

df, suma_predios, suma_materiales = datos_tabla(pesajes_df)    
print(df)
print(suma_predios)
print(suma_materiales)

                        bolson    peso
predio      material                  
AVELLANEDA  TELA        2306.0    6123
BARRACAS    Chatarra   10670.0    4136
            Mezcla B  183225.0  151714
            Vidrio B   73438.0   31966
CORTEJARENA Chatarra   12313.0   11231
            Mezcla B  232926.0  194126
            TELA       16673.0   35466
            Vidrio B   12888.0    9132
SAAVEDRA    Mezcla B  251505.0  110116
                        peso
predio      material        
AVELLANEDA  TELA        6123
BARRACAS    Chatarra    4136
            Mezcla B  151714
            Vidrio B   31966
CORTEJARENA Chatarra   11231
            Mezcla B  194126
            TELA       35466
            Vidrio B    9132
SAAVEDRA    Mezcla B  110116
               peso
predio             
AVELLANEDA     6123
BARRACAS     187816
CORTEJARENA  249955
SAAVEDRA     110116
            peso
material        
Chatarra   15367
Mezcla B  455956
TELA       41589
Vidrio B   41098


In [11]:
pesos_historico_predios(pesajes_df).show()
pesos_historico_materiales(pesajes_df).show()
pesos_historico_promedio(pesajes_df).show()
torta(pesajes_df).show()
print(f"El centro que más material reunió (en peso total) fue: {centro_principal(pesajes_df)[0]}")

         fecha       predio   bolson   peso
0   2019-12-02     BARRACAS  35554.0  72205
1   2019-12-02  CORTEJARENA   3693.0   5297
2   2019-12-02     SAAVEDRA  25223.0  10650
3   2019-12-03  CORTEJARENA  16705.0  33029
4   2019-12-03     SAAVEDRA  27228.0  11001
..         ...          ...      ...    ...
247 2020-12-28     SAAVEDRA  27865.0  14976
248 2020-12-29     BARRACAS   5658.0   4042
249 2020-12-29  CORTEJARENA  12883.0   9277
250 2020-12-30  CORTEJARENA   3634.0   2831
251 2020-12-30     SAAVEDRA  12316.0   7911

[252 rows x 4 columns]
CORTEJARENA
SAAVEDRA
nan
BARRACAS
AVELLANEDA


AttributeError: 'list' object has no attribute 'show'